In [1]:
import warnings as wns
wns.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


날씨(최저 기온, 평균 기온, 최고 기온, 강수량)와 배추 가격은 어떤 상관관계가 있는지 예측하는 AI를 만든다.  
최저 기온(minTemp), 평균 기온(avgTemp), 최고 기온(maxTemp), 강수량(rainFall)이 평균 가격(price)에 영향을 미칠 경우 가격을 예측한다. 
***
기상 정보: 기상자료개방포털(https://data.kma.go.kr)    
가격 정보: 농산물유통정보(https://www.kamis.or.kr)

In [5]:
# price_data.csv 파일을 읽어서 판다스 데이터프레임으로 저장한다.
price_data = pd.read_csv('./data/price_data.csv')
price_data.head()

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060


In [7]:
# 데이터프레임에 저장된 데이터를 텐서플로우에서 처리하기에 적합하도록 넘파이 배열 형태로 변환한다.
data = np.array(price_data, dtype=np.float32)
print(type(data))
data

<class 'numpy.ndarray'>


array([[ 2.0100100e+07, -4.9000001e+00, -1.1000000e+01,  8.9999998e-01,
         0.0000000e+00,  2.1230000e+03],
       [ 2.0100102e+07, -3.0999999e+00, -5.5000000e+00,  5.5000000e+00,
         8.0000001e-01,  2.1230000e+03],
       [ 2.0100104e+07, -2.9000001e+00, -6.9000001e+00,  1.4000000e+00,
         0.0000000e+00,  2.1230000e+03],
       ...,
       [ 2.0171228e+07,  2.9000001e+00, -2.0999999e+00,  8.0000000e+00,
         0.0000000e+00,  2.9010000e+03],
       [ 2.0171230e+07,  2.9000001e+00, -1.6000000e+00,  7.0999999e+00,
         6.0000002e-01,  2.9010000e+03],
       [ 2.0171232e+07,  2.0999999e+00, -2.0000000e+00,  5.8000002e+00,
         4.0000001e-01,  2.9010000e+03]], dtype=float32)

In [9]:
# 넘파이 배열에서 변화 요인 데이터(평균 기온, 최저 기온, 최고 기온, 강수량)로 사용할 데이터를 뽑아낸다.
xData = data[:, 1:5]
print(type(xData))
print(xData)

<class 'numpy.ndarray'>
[[ -4.9 -11.    0.9   0. ]
 [ -3.1  -5.5   5.5   0.8]
 [ -2.9  -6.9   1.4   0. ]
 ...
 [  2.9  -2.1   8.    0. ]
 [  2.9  -1.6   7.1   0.6]
 [  2.1  -2.    5.8   0.4]]


In [10]:
# 넘파이 배열에서 결과(평균 가격)로 사용할 데이터를 뽑아낸다.
yData = data[:, 5:] # 2차원 데이터로 뽑아낸다. => 변화 요인이 2차원이므로 결과도 2차원 형태로 뽑아내야 한다.
print(type(yData))
print(yData)

<class 'numpy.ndarray'>
[[2123.]
 [2123.]
 [2123.]
 ...
 [2901.]
 [2901.]
 [2901.]]


In [11]:
# 뽑아낸 데이터를 텐서플로우로 처리하기 위해서 placeholder를 만든다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 4]) # 변화 요인(평균 기온, 최저 기온, 최대 기온, 강수량)
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # 가격(실제값)

In [12]:
# 다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화한다. (가중치)
a = tf.Variable(tf.random_uniform([4, 1]), dtype=tf.float32) # 기울기, 4행 1열의 난수를 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)    # y절편

In [13]:
# 행렬의 곱셈 연산을 이용해서 다변인 선형 회귀 보델의 가설 식을 세운다. => 예측값을 계산하는 식
y = tf.matmul(X, a) + b # 예측값
# 오차 함수를 만든다. => 예측값(y)과 실제값(Y)의 편차의 제곱에 대한 평균 => 평균 제곱법
loss = tf.reduce_mean(tf.square(y - Y))
# 경사하강법 알고리즘을 사용해서 오차 함수의 결과를 최소로 하는 식을 찾는다.
gradient_descent = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)

In [16]:
# 학습시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(100001):
    loss_, y_, _ = sess.run([loss, y, gradient_descent], feed_dict={X: xData, Y: yData})
    if i % 5000 == 0:
        print('Epoch: {0:6d}, loss: {1:12.3f}, price: {2:8.2f}'.format(i, loss_, y_[0][0]))
        
# ===== 학습완료

# 학습이 완료되면 학습된 학습 모델을 디스크에 저장한다.
# Saver() 메소드로 텐서플로우에서 학습된 모델을 디스크에 저장 또는 불러올 때 사용하는 객체를 생성한 메소드
saver = tf.train.Saver()
# save() 메소드로 텐서플로우에서 학습된 모델을 디스크에 저장한다.
save_path = saver.save(sess, './data/saved.cpkt')
print('학습된 모델을 저장했습니다.')

Epoch:      0, loss: 12472502.000, price:    -3.90
Epoch:   5000, loss:  2254646.750, price:  2581.54
Epoch:  10000, loss:  2228483.000, price:  2584.42
Epoch:  15000, loss:  2204128.750, price:  2587.43
Epoch:  20000, loss:  2181440.250, price:  2590.60
Epoch:  25000, loss:  2160287.500, price:  2593.88
Epoch:  30000, loss:  2140551.750, price:  2597.24
Epoch:  35000, loss:  2122124.500, price:  2600.70
Epoch:  40000, loss:  2104905.000, price:  2604.23
Epoch:  45000, loss:  2088803.000, price:  2607.86
Epoch:  50000, loss:  2073737.250, price:  2611.55
Epoch:  55000, loss:  2059629.125, price:  2615.27
Epoch:  60000, loss:  2046410.250, price:  2619.04
Epoch:  65000, loss:  2034016.000, price:  2622.83
Epoch:  70000, loss:  2022389.125, price:  2626.64
Epoch:  75000, loss:  2011474.125, price:  2630.52
Epoch:  80000, loss:  2001221.375, price:  2634.36
Epoch:  85000, loss:  1991589.375, price:  2638.20
Epoch:  90000, loss:  1982528.875, price:  2642.07
Epoch:  95000, loss:  1974003.2

In [17]:
from IPython.display import Image
# 이미지